In [45]:
import random

# Define problem instance
courses=[] #the courses list randomly generated 
numCourses=random.randint(5,30) 
for i in range(numCourses):
    courses.append("C"+str(i+1))
    
print(courses)

time_slots=[] #the time slots list randomly generated
timeslot=random.randint(3,10)
for i in range(timeslot):
    time_slots.append("T"+str(i+1))
    
print(time_slots)

examHalls=random.randint(2,5)
exam_halls=[] #the exam halls list randomly generated
for i in range (examHalls):
    exam_halls.append("H"+str(i+1))
print(exam_halls)

hall_max_hours = 6 #the maximum number of halls for a hall
conflicting_students = { #the dictionary containg key value as the courses and the value as number of students
    ("C1", "C2"): 10,
    ("C1", "C4"): 5,
    ("C2", "C5"): 7,
    ("C3", "C4"): 12,
    ("C4", "C5"): 8,
}

# 1. Representation
# A solution is represented as a list of tuples (course, time_slot, exam_hall)
# solution = [('C1', 'T1', 'H1'), ('C2', 'T2', 'H1'), ('C3', 'T3', 'H2'), ('C4', 'T1', 'H2'), ('C5', 'T2', 'H2')]

# 2. Fitness function
def fitness_score(solution):
    score = 0
    hall_hours = {hall: 0 for hall in exam_halls}

    for exam in solution:
        course, time_slot, exam_hall = exam
        hall_hours[exam_hall] += 1

        for other_exam in solution:
            other_course, other_time_slot, other_exam_hall = other_exam
            if course != other_course:
                if time_slot == other_time_slot:
                    if (course, other_course) in conflicting_students:
                        score += 100 * conflicting_students[(course, other_course)]
                    elif (other_course, course) in conflicting_students:
                        score += 100 * conflicting_students[(other_course, course)]

    for hall, hours in hall_hours.items():
        if hours > hall_max_hours:
            score += 10 * (hours - hall_max_hours)

    return score


# 3. Genetic Algorithm
def generate_random_solution():
    return [
        (course, random.choice(time_slots), random.choice(exam_halls))
        for course in courses
    ]


def initialize_population(population_size):
    return [generate_random_solution() for _ in range(population_size)]


def tournament_selection(population, tournament_size):
    candidates = random.sample(population, tournament_size)
    return min(candidates, key=fitness_score)


def single_point_crossover(parent1, parent2):
    crossover_point = random.randint(1, len(courses) - 1)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    return offspring1, offspring2


def mutate(solution, mutation_probability):
    mutated = solution.copy()
    for i in range(len(courses)):
        if random.random() < mutation_probability:
            mutated[i] = (
                courses[i],
                random.choice(time_slots),
                random.choice(exam_halls),
            )
    return mutated


def genetic_algorithm(population_size,generations,tournament_size,crossover_probability,
    mutation_probability,
):
    population = initialize_population(population_size)
    for _ in range(generations):
        new_population = []
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)

            if random.random() < crossover_probability:
                offspring1, offspring2 = single_point_crossover(parent1, parent2)
            else:
                offspring1, offspring2 = parent1, parent2

            offspring1 = mutate(offspring1, mutation_probability)
            offspring2 = mutate(offspring2, mutation_probability)

            new_population.extend([offspring1, offspring2])

        population = new_population

    best_solution = min(population, key=fitness)
    return best_solution, fitness(best_solution)

population_size = 100
generations = 100
tournament_size = 5
crossover_probability = 0.8
mutation_probability = 0.1

best_solution, best_fitness = genetic_algorithm(population_size,generations,tournament_size,
    crossover_probability,
    mutation_probability,
)

print("solution:", best_solution)
print("Fitness score:", best_fitness)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19']
['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8']
['H1', 'H2', 'H3', 'H4', 'H5']
solution: [('C1', 'T5', 'H1'), ('C2', 'T2', 'H2'), ('C3', 'T4', 'H3'), ('C4', 'T2', 'H4'), ('C5', 'T5', 'H2'), ('C6', 'T4', 'H5'), ('C7', 'T4', 'H1'), ('C8', 'T2', 'H1'), ('C9', 'T5', 'H2'), ('C10', 'T5', 'H1'), ('C11', 'T8', 'H4'), ('C12', 'T6', 'H1'), ('C13', 'T2', 'H4'), ('C14', 'T6', 'H3'), ('C15', 'T2', 'H4'), ('C16', 'T8', 'H2'), ('C17', 'T4', 'H2'), ('C18', 'T7', 'H3'), ('C19', 'T3', 'H3')]
Fitness score: 0
